In [2]:
#!/usr/bin/env python
# encoding: utf-8

import csv
import ctypes
import json
import os
import re
import sys
import numpy as np
import pandas as pd
import shutil
import time
import zipfile
from os import listdir as ls
from os.path import basename as bname
from os.path import dirname as dname
from os.path import expanduser as xpu
from os.path import join
from pandas import DataFrame as df
from cimaq_utils import flatten
from cimaq_utils import loadimages
# ctypes.windll.kernel32.SetFileAttributesW(path, 2)

def NEW_extract_events2(
    zipdir="~/../../data/cisl/DATA/cimaq_03-19/derivatives/CIMAQ_fmri_memory/data/task_files/zipped_eprime",
    outdir="~/extracted_eprime2"):
    
    # Initializing BIDS compliant participants sheet
    outdir, zipdir = xpu(outdir), xpu(zipdir)
    participants = df([("sub-"+bname(zfile).split("_")[0], bname(zfile).split("_")[0],
                        bname(zfile).split("_")[1], bname(zfile), zfile,
                        join(outdir, "sub-"+bname(zfile).split("_")[0]))
                       for zfile in loadimages(zipdir) if zfile.endswith(".zip")],
                      columns=["sub-ID", "pscid", "dccid", "zfilename",
                               "zipdirpath", "extdirpath"]).set_index("sub-ID").sort_index()
    # Extract files
    os.makedirs(outdir, exist_ok=True)
    for row in participants.iterrows():
        archv = zipfile.ZipFile(row[1]["zipdirpath"], 'r')
        archv.extractall(row[1]["extdirpath"])
        archv.close()
    
    # Move files to BIDS compliant directory
    [[shutil.move(file, join(outdir, row[0], bname(file)))
      for file in loadimages(row[1]["extdirpath"]) if "CIMAQ" in bname(file)]
     for row in participants.iterrows()]
    participants["sheetpaths"] = sorted([join(outdir, row[0], file) for file in ls(join(outdir, row[0]))
                                         if "CIMAQ" in file and file.endswith(".txt")
                                         for row in participants.iterrows()])
    
    [os.makedirs(join(outdir, row[0], "."+row[0]+"_"+"archives"), exist_ok=True)
     for item in ls(join(outdir, row[0]))]
    [[shutil.move(join(outdir, row[0], item),
                  join(outdir, row[0], row[0]+"_"+"archives", item))
     for item in ls(join(outdir, row[0])) if os.path.isdir(join(outdir, row[0], item))]
    for row in participants.iterrows()]
     
    # Clear empty archives
    for topdir, folders, files in os.walk(outdir, topdown=False):
        for folder in folders:
            if "__MACOSX" in join(topdir, folder):
                shutil.rmtree(join(topdir, folder))
    participants.to_csv(join(outdir, "participants.tsv"), sep='\t')
    return participants

def main():
    participants = NEW_extract_events2()

if __name__ == '__main__':
    main()

ValueError: Length of values does not match length of index